In [2]:
x = ["April 22,2019", "March 10,2019", "June 09,2019", "March 11,2021", "March 12,2019"]
y = ["2019-04-22", "2019-03-10", "2019-06-09", "2021-03-11", "2019-03-12"]

In [ ]:
model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=len(x) + 1,
                           output_dim=embedding_size,
                           input_shape=[None]),
    keras.layers.LSTM(128)
])

decoder = keras.models.Sequential([
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.Dense(len())
])

First version: a very basic seq2seq model

Second version: feeding the shifted targets to the decoder (teacher forcing)

Third version: using TF-Addons's seq2seq implementation

Fourth version: using TF-Addons's seq2seq implementation with a scheduled sampler

Fifth version: using TFA seq2seq, the Keras subclassing API and attention mechanisms

> 첫 번째 버전: 매우 기본적인 seq2seq 모델
이 모델은 기본적인 seq2seq 모델입니다. 인코더는 입력 시퀀스를 입력 받고 마지막 시간 단계에서 컨텍스트 벡터를 출력하며, 디코더는 이전 시간 단계의 출력을 현재 시간 단계의 입력으로 사용하여 시퀀스를 생성합니다.

인코더는 입력 문자열을 임베딩하고, LSTM 레이어를 사용하여 입력 시퀀스를 인코딩합니다.
전체 모델은 인코더, RepeatVector 레이어(출력 시퀀스의 길이와 맞추기 위해 입력 시퀀스를 반복), 디코더로 구성됩니다.

> 두 번째 버전: 디코더에 시프트된 타겟을 입력하는 teacher forcing
이 모델은 두 번째 버전입니다. 이 모델은 첫 번째 버전과 거의 동일하지만, 디코더는 이전 시간 단계에서의 출력 대신 시프트된 타겟 시퀀스를 현재 시간 단계의 입력으로 사용합니다. 이 방법을 "teacher forcing" 이라고 합니다. 이렇게 하면 디코더가 예측한 출력을 다음 시간 단계에서의 입력으로 사용하지 않고 실제 타겟 값을 사용하므로 학습이 더 빠르고 안정적으로 이루어집니다.

> 세 번째 버전: TF-Addons의 seq2seq 구현 사용
이 모델은 TF-Addons의 seq2seq 구현을 사용합니다. 이 구현은 첫 번째와 두 번째 모델과 비슷하지만, 좀 더 일반화된 구현을 제공합니다. 또한 이 구현에서는 입력과 출력 시퀀스의 길이를 일치시키기 위해 패딩이 사용됩니다.

> 네 번째 버전: 스케줄된 샘플러를 사용한 TF-Addons의 seq2seq 구현
이 모델은 TF-Addons의 seq2seq 구현을 사용합니다. 이 구현에서는 스케줄된 샘플링을 사용하여 디코더의 입력을 생성합니다. 이 방법을 사용하면 디코더가 일정 비율로 이전 시간 단계의 출력과 타겟 값을 사용하므로, 더 정확한 예측을 할 수 있습니다.

> 다섯 번째 버전: TFA seq2seq, Keras 서브클래싱 API 및 어텐션 메커니즘 사용
이 모델은 TFA seq2seq, Keras 서브클래싱 API 및 어텐션 메커니즘을 사용합니다. 이 모델은 이전 버전에서 사용된 모든 기술을 결합하며, 어텐션 메커니즘을 사용하여 디코더가 인코더의 출력 중에서 어떤 부분에 더 집중해야있는지 결정합니다. 이 모델은 Keras의 서브클래싱 API를 사용하여 모델을 정의하며, TFA seq2seq 구현을 사용하여 학습을 수행합니다. 이 모델은 이전 버전에 비해 더 높은 성능을 제공할 수 있습니다.

In [8]:

from datetime import date
import numpy as np
import tensorflow as tf

# cannot use strftime()'s %B format since it depends on the locale
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()

    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y

def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))
    
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
OUTPUT_CHARS = "0123456789-"

def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor() # using 0 as the padding token ID

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

sos_id = len(OUTPUT_CHARS) + 1

def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)

X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/gahyunson/opt/anaconda3/envs/mywork/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/0s/knzc6bzs1l9dgw6x4k24t9740000gn/T/ipykernel_64292/1662734598.py", line 20, in <module>
    def date_str_to_ids(date_str, chars=INPUT_CHARS):
NameError: name 'INPUT_CHARS' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/gahyunson/opt/anaconda3/envs/mywork/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/gahyunson/opt/anaconda3/envs/mywork/lib/python3.7/site-packages/IPython/core/ultratb.py"

In [ ]:
X_train_decoder

In [7]:
import tensorflow as tf; print(tf.__version__)

1.15.0
